##### the concept of printing the statements to validate if the statements are executed correctly or if some error has occurred. But printing is not a good idea. It may solve your issues for simple scripts but for complex scripts, the printing approach will fail.
##### Python has a built-in module logging which allows writing status messages to a file or any other output streams. The file can contain information       on which part of the code is executed and what problems have arisen.  



In [1]:
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def divide(a, b):
    logging.debug(f"Received a={a}, b={b}")
    if b == 0:
        logging.error("Attempted to divide by zero")
        return None
    result = a / b
    logging.info(f"Result of division: {result}")
    return result

# Example usage
divide(10, 2)
divide(10, 0)


2024-12-30 12:06:05,107 - DEBUG - Received a=10, b=2
2024-12-30 12:06:05,108 - INFO - Result of division: 5.0
2024-12-30 12:06:05,109 - DEBUG - Received a=10, b=0
2024-12-30 12:06:05,110 - ERROR - Attempted to divide by zero


In [5]:
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(filename)s:%(lineno)d - %(levelname)s - %(message)s'
)

def my_function():
    logging.debug("This is a debug message")

my_function()


In [4]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y/%m/%d %I:%M:%S %p'
)

logging.info("Application started")


2024/12/30 12:31:59 PM - Application started


In [2]:
import logging
import random
import time

# Configure the root logger
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("pos.log"),  # Log to a file
        logging.StreamHandler()          # Log to the console
    ]
)

# Create separate loggers for different parts of the system
order_logger = logging.getLogger("OrderProcessor")
payment_logger = logging.getLogger("PaymentProcessor")
audit_logger = logging.getLogger("AuditLogger")


class InvalidOrderError(Exception):
    """Custom exception for invalid orders."""


class OrderProcessor:
    """Processes orders for the POS system."""
    
    @staticmethod
    def validate_order(order):
        """Validate the order."""
        order_logger.debug(f"Validating order: {order}")
        if not order.get("id") or not order.get("items"):
            order_logger.error("Invalid order: Missing ID or items")
            raise InvalidOrderError("Order is missing required fields")

        for item in order["items"]:
            if not item.get("name") or item.get("price") <= 0:
                order_logger.error(f"Invalid item in order: {item}")
                raise InvalidOrderError(f"Item {item} is invalid")
        
        order_logger.info(f"Order {order['id']} validated successfully")

    @staticmethod
    def calculate_total(order):
        """Calculate the total cost of the order."""
        total = sum(item["price"] * item["quantity"] for item in order["items"])
        order_logger.debug(f"Calculated total for order {order['id']}: {total}")
        return total


class PaymentProcessor:
    """Handles payment processing for the POS system."""
    
    @staticmethod
    def process_payment(order_id, amount):
        """Simulate payment processing."""
        payment_logger.debug(f"Processing payment for order {order_id}, amount: {amount}")
        if random.choice([True, False]):  # Simulate payment failures randomly
            payment_logger.error(f"Payment failed for order {order_id}")
            raise Exception("Payment processing failed")
        payment_logger.info(f"Payment successful for order {order_id}")


def main():
    """Main function to simulate the POS system."""
    orders = [
        {"id": 1, "items": [{"name": "Laptop", "price": 1200, "quantity": 1}, {"name": "Mouse", "price": 25, "quantity": 2}]},
        {"id": 2, "items": [{"name": "Headphones", "price": 100, "quantity": 3}]},
        {"id": None, "items": [{"name": "Keyboard", "price": 50, "quantity": 1}]},  # Invalid order (missing ID)
        {"id": 3, "items": [{"name": "Monitor", "price": -200, "quantity": 1}]},    # Invalid order (negative price)
    ]

    for order in orders:
        try:
            # Validate the order
            OrderProcessor.validate_order(order)

            # Calculate total
            total = OrderProcessor.calculate_total(order)

            # Process payment
            PaymentProcessor.process_payment(order["id"], total)

            # Log successful transaction for auditing
            audit_logger.info(f"Order {order['id']} processed successfully. Total: {total}")

        except InvalidOrderError as e:
            order_logger.warning(f"Order {order.get('id', 'UNKNOWN')} validation failed: {e}")
        except Exception as e:
            payment_logger.critical(f"Critical failure for order {order.get('id', 'UNKNOWN')}: {e}")
        finally:
            time.sleep(1)  # Simulate some delay between orders


if __name__ == "__main__":
    main()


2024-12-30 13:55:14,024 - OrderProcessor - DEBUG - Validating order: {'id': 1, 'items': [{'name': 'Laptop', 'price': 1200, 'quantity': 1}, {'name': 'Mouse', 'price': 25, 'quantity': 2}]}
2024-12-30 13:55:14,025 - OrderProcessor - INFO - Order 1 validated successfully
2024-12-30 13:55:14,026 - OrderProcessor - DEBUG - Calculated total for order 1: 1250
2024-12-30 13:55:14,027 - PaymentProcessor - DEBUG - Processing payment for order 1, amount: 1250
2024-12-30 13:55:14,028 - PaymentProcessor - ERROR - Payment failed for order 1
2024-12-30 13:55:14,029 - PaymentProcessor - CRITICAL - Critical failure for order 1: Payment processing failed
2024-12-30 13:55:15,031 - OrderProcessor - DEBUG - Validating order: {'id': 2, 'items': [{'name': 'Headphones', 'price': 100, 'quantity': 3}]}
2024-12-30 13:55:15,036 - OrderProcessor - INFO - Order 2 validated successfully
2024-12-30 13:55:15,041 - OrderProcessor - DEBUG - Calculated total for order 2: 300
2024-12-30 13:55:15,046 - PaymentProcessor - DE

In [2]:
import logging

class LoggerDemo:
    def sample_logger(self):
        # Create a logger
        logger = logging.getLogger('demolog')

        # Remove any existing handlers (Fixes duplicate logging)
        if logger.hasHandlers():
            logger.handlers.clear()

        logger.setLevel(logging.DEBUG)  # Set logging level

        # Create a stream handler to print logs to the console
        sh = logging.StreamHandler()
        fh=logging.FileHandler("logs_test.log")
        # Define log format
        log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        formatter = logging.Formatter(fmt=log_format)

        # Attach formatter to the stream handler
        sh.setFormatter(formatter)
        fh.setFormatter(formatter)
        # Add the handler to the logger
        # logger.addHandler(sh)
        logger.addHandler(fh)
        logger.info("This is an INFO statement")
        logger.debug("This is a DEBUG statement")
        logger.warning("This is a WARNING statement")
        logger.error("This is an ERROR statement")
        logger.critical("This is a CRITICAL statement")

# Create instance and call function
ld = LoggerDemo()
ld.sample_logger()
